In [62]:
import numpy as np
import plotly.graph_objects as go

from orbslam_results import ORBSLAMTrajectoryProcessor
from utils import create_scattermapbox, plot

# processor = ORBSLAMTrajectoryProcessor('~/orbslam-out/vabadusepst')
processor = ORBSLAMTrajectoryProcessor('~/orbslam-out/oldtown')

trace_colors = {
   'gps': 'blue',
   'slam': 'red',
   'loc': ['dodgerblue', 'olive', 'orange']
}

fig_slam = go.Figure()

fig_slam.add_trace(
   go.Scatter(
      x=processor.orbslam.mapping.trajectory[:, 0], 
      y=processor.orbslam.mapping.trajectory[:, 2], 
      mode='markers', 
      name='SLAM (mapping)',
      marker=dict(color=trace_colors['slam'])
    )
)



for name, loc in processor.orbslam.localization.items():
    if (loc.trajectory.size > 0):
      fig_slam.add_trace(
         go.Scatter(
            x=loc.trajectory[:, 0], 
            y=loc.trajectory[:, 2], 
            mode='markers',
            marker=dict(color=trace_colors['loc'][name])
         )
      )

fig_slam.update_layout(yaxis=dict(scaleanchor="x", scaleratio=1), height=1200)
fig_slam.show()

In [58]:
processor.fit()

Training model with (1522, 3)
Target points: (1522, 3)
Resized source:  (1522, 3)
Resized target:  (1522, 3)
Epoch 1/400
191/191 [==============================] - 0s 389us/step - loss: 0.2943 - lr: 0.0010
Epoch 2/400
191/191 [==============================] - 0s 369us/step - loss: 0.0280 - lr: 0.0010
Epoch 3/400
191/191 [==============================] - 0s 362us/step - loss: 0.0249 - lr: 0.0010
Epoch 4/400
191/191 [==============================] - 0s 355us/step - loss: 0.0235 - lr: 0.0010
Epoch 5/400
191/191 [==============================] - 0s 341us/step - loss: 0.0223 - lr: 0.0010
Epoch 6/400
191/191 [==============================] - 0s 361us/step - loss: 0.0216 - lr: 0.0010
Epoch 7/400
191/191 [==============================] - 0s 360us/step - loss: 0.0206 - lr: 0.0010
Epoch 8/400
191/191 [==============================] - 0s 415us/step - loss: 0.0201 - lr: 0.0010
Epoch 9/400
191/191 [==============================] - 0s 370us/step - loss: 0.0194 - lr: 0.0010
Epoch 10/400
191/1

In [61]:
from utils import umeyama_alignment


center_lat = np.mean(processor.orbslam.mapping.trajectory_gt_wgs[:, 0])
center_lon = np.mean(processor.orbslam.mapping.trajectory_gt_wgs[:, 1])

R, t, c = umeyama_alignment(processor.orbslam.mapping.trajectory.T,
                            processor.orbslam.mapping.trajectory_gt_utm.T,
                            True)
aligned_slam_trajectory_utm = np.array([t + c * R @ p for p in processor.orbslam.mapping.trajectory])
aligned_slam_trajectory_wgs = np.array([processor.orbslam.mapping.utm2wgs.transform(
    p[0], p[1], p[2])for p in aligned_slam_trajectory_utm])

traces = [
    create_scattermapbox(processor.orbslam.mapping.trajectory_gt_wgs, 'GPS (ground truth)', color=trace_colors['gps'], mode='lines'),
    create_scattermapbox(processor.orbslam.mapping.trajectory_wgs, 'fitted SLAM (mapping)', color=trace_colors['slam'], mode='lines'),
    create_scattermapbox(aligned_slam_trajectory_wgs, 'Umeyama SLAM (mapping)', color='green', mode='lines'),
]

for name, localization in processor.orbslam.localization.items():
    traces.append(create_scattermapbox(localization.trajectory_wgs, f'fitted loc_{name}', color=trace_colors['loc'][name]))

plot(traces,(center_lat, center_lon))
